In [1]:
import torch
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

Испльзование заранее предобученной модели называется transfer learning. Оно бывает двух видов  

`finetuning` - во время дообучения обновляются все веса, которые есть в модели  

`feature extraction` - во время дообучения обновляются только веса последнего слоя. По сути  
   мы используем предобученную нейросеть, как готовый извлекатель признаков и лишь по новому  
   их комбинируем

для примера возьмем resnet18

Начнем с функции обучения 

In [2]:
import copy

def train(model, dataloaders, optim, lossFunc, epochs, device):
    
    #эмулируем чекпоинты
    bestWeights = copy.deepcopy(model.state_dict())
    bestAcc = 0.0
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)
        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            currentLoss = 0.0
            currentCorrects = 0

            for images, labels in dataloaders[phase]:
                images = images.to(device)
                labels = labels.to(device)
                
                optim.zero_grad()
                # то же самое, что и with torch.no_grad(), но включается по булу
                with torch.set_grad_enabled(phase == "train"):
                    out = model(images)
                    loss = lossFunc(out, labels)
                    _, preds = torch.max(out, 1)
                    if phase == "train":
                        loss.backward()
                        optim.step()
                
                currentLoss += loss.item() * images.size(0)
                currentCorrects += torch.sum(preds == labels.data)

            epochLoss = currentLoss / len(dataloaders[phase].dataset)
            epochAcc = currentCorrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epochLoss, epochAcc))

            if phase == 'val' and epochAcc > bestAcc:
                bestAcc = epochAcc
                bestWeights = copy.deepcopy(model.state_dict())
    print('Best val Acc: {:4f}'.format(bestAcc))
    model.load_state_dict(bestWeights)
    return model

